In [1]:
table_name = "anti_frod"

In [2]:
import psycopg2
import pandas as pd

# Настройка соединения с базой данных PostgreSQL
conn = psycopg2.connect(
    user="postgres",
    password="1",
    host="localhost",
    port="5435",
    database="postgres"
)

# Используем pandas для загрузки данных в DataFrame
df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)

# Закрываем соединение
conn.close()


C:\Users\idukh\AppData\Local\Temp\ipykernel_32548\4040667839.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)


In [3]:
import psycopg2

def find_columns(table_name, conn):
    # Ключевые слова для поиска времени и типа события
    event_time_keywords = ['event', 'time', 'timestamp', 'date']
    event_type_keywords = ['event', 'type', 'category', 'action']

    def column_score(column_name, data_type, keywords, target_types):
        score = 0
        column_name_lower = column_name.lower()

        # Увеличиваем вес для точных ключевых слов
        for keyword in keywords:
            if keyword in column_name_lower:
                score += 2
            if column_name_lower.startswith(keyword) or column_name_lower.endswith(keyword):
                score += 1

        # Увеличиваем вес, если тип данных совпадает с целевым
        if data_type in target_types:
            score += 3
        return score

    # Подключение и выполнение запроса для получения столбцов таблицы с их типами данных
    with conn.cursor() as cur:
        cur.execute(f"""
            SELECT column_name, data_type 
            FROM information_schema.columns 
            WHERE table_name = %s
        """, (table_name,))
        
        columns = cur.fetchall()

    # Поиск столбца для времени события
    event_time_column = max(
        columns, 
        key=lambda col: column_score(col[0], col[1], event_time_keywords, ['Timestamp', 'time'])
    )

    # Поиск столбца для типа события
    event_type_column = max(
        columns, 
        key=lambda col: column_score(col[0], col[1], event_type_keywords, ['text'])
    )

    return event_time_column[0], event_type_column[0]

# Пример подключения к базе данных
conn = psycopg2.connect(
    user="postgres",
    password="1",
    host="localhost",
    port="5435",
    database="postgres"
)

# Поиск столбцов
event_time_column, event_type_column = find_columns(table_name, conn)
print(f"Event Time Column: {event_time_column}")
print(f"Event Type Column: {event_type_column}")

# Закрытие соединения
conn.close()


Event Time Column: event_time
Event Type Column: event_type


In [4]:
time_1 = False
time_6 = False

In [5]:
if event_time_column != '' or event_type_column != '':
    time_1 = True

In [6]:
numeric_columns = df.select_dtypes(include='number')

correlation_matrix = numeric_columns.corr()
final_query = []
threshold = 0.5
for col1 in correlation_matrix.columns:
    for col2 in correlation_matrix.columns:
        if col1 != col2 and correlation_matrix[col1][col2] > threshold:
            time_6 = True

In [7]:
import json

# Создаем словарь с данными, которые нужно записать
data = {
    "table_name": f"{table_name}"
}

# Записываем данные в JSON файл
with open("name_table.json", "w") as json_file:
    json.dump(data, json_file, ensure_ascii=False, indent=4)


In [8]:
import nbformat
from nbclient import NotebookClient

def run_notebook(notebook_path):
    # Читаем файл .ipynb с указанием кодировки utf-8
    with open(notebook_path, encoding="utf-8") as f:
        notebook = nbformat.read(f, as_version=4)
    
    # Инициализируем клиент для выполнения
    client = NotebookClient(notebook)
    
    # Выполняем все ячейки
    client.execute()

if time_1 and time_6:
    run_notebook("cod_create_dash_Anti_Frod_time.ipynb")
elif time_1 and not time_6:
    run_notebook("cod_ctreate_dash_Anti_Frod_time_no_time_6.ipynb")
elif time_6 and not time_1:
    run_notebook("cod_ctreate_dash_Anti_Frod_time_no_time_1.ipynb")
else:
    run_notebook("cod_ctreate_dash_Anti_Frod_time_no_time_1_no_time_6.ipynb")
